# 🔥 VTrackAI Studio - E2E GPU Setup

**Prerequisites:**
1. Runtime → Change runtime type → **T4 GPU**
2. Sidebar → Secrets → Add `HF_TOKEN` (from huggingface.co/settings/tokens)
3. Run all cells (Ctrl+F9)

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: SETUP (Run once per session, ~3-5 min)
# ═══════════════════════════════════════════════════════════════════════════════
import torch, os, sys

# GPU Check
print("="*60)
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ NO GPU! Go to Runtime → Change runtime type → T4 GPU")
    sys.exit(1)
print("="*60)

# Clean start
%cd /content
!rm -rf /content/sam3 /content/vtrack-ai-studio

# Install dependencies
print("\n📦 Installing dependencies...")
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q fastapi uvicorn[standard] python-multipart huggingface_hub requests decord
!pip install -q numpy==1.26.4  # SAM3 requires numpy<2

# Node.js 18 via nvm
print("\n📦 Installing Node.js 18...")
!curl -so- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.7/install.sh | bash > /dev/null 2>&1
import subprocess
result = subprocess.run(["bash", "-lc", '''
  export NVM_DIR="$HOME/.nvm"
  [ -s "$NVM_DIR/nvm.sh" ] && . "$NVM_DIR/nvm.sh"
  nvm install 18 > /dev/null 2>&1
  nvm use 18 > /dev/null 2>&1
  echo "Node $(node -v), npm $(npm -v)"
'''], capture_output=True, text=True)
print(f"✅ {result.stdout.strip()}")

# Clone SAM3
print("\n📦 Cloning SAM3...")
%cd /content
!git clone -q https://github.com/facebookresearch/sam3
!pip install -e /content/sam3 -q 2>/dev/null

# Clone VTrackAI
print("📦 Cloning VTrackAI...")
!git clone -q https://github.com/Amitro123/vtrack-ai-studio
%cd /content/vtrack-ai-studio/backend
!pip install -r requirements.txt -q 2>/dev/null

# HuggingFace login + checkpoint
print("\n🔐 Downloading SAM3 checkpoint...")
from google.colab import userdata
from huggingface_hub import login, hf_hub_download

HF_TOKEN = userdata.get("HF_TOKEN")
login(token=HF_TOKEN, add_to_git_credential=False)

os.makedirs("/content/vtrack-ai-studio/backend/checkpoints/sam3", exist_ok=True)
checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3",
    filename="sam3.pt",
    local_dir="/content/vtrack-ai-studio/backend/checkpoints/sam3",
    token=HF_TOKEN,
)

# Create symlinks
sam3_pt = "/content/vtrack-ai-studio/backend/checkpoints/sam3/sam3.pt"
for dst in [
    "/content/vtrack-ai-studio/backend/checkpoints/sam3/sam3_hiera_large.pt",
]:
    if os.path.exists(dst): os.remove(dst)
    os.symlink(sam3_pt, dst)

print(f"✅ Checkpoint ready: {checkpoint_path}")
print("\n" + "="*60)
print("✅ SETUP COMPLETE! Run Cell 2 to start backend.")
print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: START BACKEND (FastAPI on port 8000)
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess, time, requests, os

# Kill any existing backend
!pkill -9 -f uvicorn 2>/dev/null || true
time.sleep(2)

# Start backend in background
print("🚀 Starting FastAPI backend...")
log_file = open("/content/backend.log", "w")
proc = subprocess.Popen(
    ["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=log_file,
    stderr=subprocess.STDOUT,
    cwd="/content/vtrack-ai-studio/backend",
)

# Wait for backend with retries
for i in range(20):
    if proc.poll() is not None:
        print(f"❌ Backend crashed! Logs:")
        !cat /content/backend.log
        break
    try:
        r = requests.get("http://localhost:8000/api/health", timeout=3)
        health = r.json()
        print("\n" + "="*60)
        print(f"✅ Backend running on port 8000")
        print(f"   SAM3: {'✅ Available' if health.get('sam3_available') else '❌ Not loaded'}")
        print(f"   Device: {health.get('device', 'unknown')}")
        print("="*60)
        break
    except:
        print(f"⏳ Waiting... ({i+1}/20)")
        time.sleep(3)
else:
    print("❌ Backend failed to start. Logs:")
    !cat /content/backend.log

print("\n✅ Run Cell 3 to start frontend.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: START FRONTEND (Vite + Cloudflare tunnel)
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess, time

# Start Vite frontend
print("🚀 Starting Vite frontend...")
bash_cmd = '''
cd /content/vtrack-ai-studio
export NVM_DIR="$HOME/.nvm"
[ -s "$NVM_DIR/nvm.sh" ] && . "$NVM_DIR/nvm.sh"
nvm use 18 > /dev/null 2>&1
echo 'VITE_API_URL=http://localhost:8000' > .env
npm install --silent 2>/dev/null
nohup npm run dev -- --host 0.0.0.0 --port 4173 > /content/vite.log 2>&1 &
echo "Vite PID: $!"
'''
result = subprocess.run(["bash", "-lc", bash_cmd], capture_output=True, text=True)
print(result.stdout)

# Wait for Vite to start
time.sleep(10)

# Start Cloudflare tunnel
print("🌐 Creating Cloudflare tunnel...")
!pip install -q cloudflared 2>/dev/null
!pkill cloudflared 2>/dev/null || true
!nohup cloudflared tunnel --url http://localhost:4173 > /content/tunnel.log 2>&1 &

# Wait and extract URL
time.sleep(8)
import re
try:
    with open("/content/tunnel.log") as f:
        log = f.read()
    urls = re.findall(r'https://[\w-]+\.trycloudflare\.com', log)
    if urls:
        print("\n" + "="*60)
        print("🎉 VTrackAI Studio is LIVE!")
        print(f"\n   👉 {urls[0]}")
        print("\n   Open this URL in your browser.")
        print("="*60)
    else:
        print("⏳ Waiting for tunnel URL...")
        !cat /content/tunnel.log | grep -o 'https://.*trycloudflare.com' | head -1
except:
    !cat /content/tunnel.log | grep -o 'https://.*trycloudflare.com' | head -1

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: DEBUG / LOGS (Run if needed)
# ═══════════════════════════════════════════════════════════════════════════════
print("=== Backend Logs ===")
!tail -20 /content/backend.log 2>/dev/null || echo "No backend log"

print("\n=== Vite Logs ===")
!tail -10 /content/vite.log 2>/dev/null || echo "No vite log"

print("\n=== Tunnel Logs ===")
!tail -10 /content/tunnel.log 2>/dev/null || echo "No tunnel log"

print("\n=== Process Status ===")
!ps aux | grep -E 'uvicorn|node|cloudflared' | grep -v grep